<a href="https://colab.research.google.com/github/dudaribeiro7/Trabalho_Pesquisa_Operacional/blob/main/Pesquisa_Operacional_Trabalho_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Leitura dos Dados**

## **Acesso à Planilha 'Pesquisa Operacional - Trabalho 2' no Google Drive**

In [ ]:
!pip install gspread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
# Acesso à planilha no google sheets
planilha = gc.open('Pesquisa Operacional - Trabalho 2')

In [ ]:
print('Selecione o curso desejado: ')
print('1. Engenharia Computacional')
print('2. Direito')
print('3. Serviço Social')

curso = int(input('Curso: '))

Selecione o curso desejado: 
1. Engenharia Computacional
2. Direito
3. Serviço Social
Curso: 1


In [ ]:
if curso == 1:
  # Acesso à página Disciplinas EngComp: Pré-Requisitos
  preReq = planilha.get_worksheet(0)

  # Acesso à página Disciplinas EngComp
  infoDisc = planilha.get_worksheet(1)

  # Acesso à página Períodos EngComp
  infoPeriod = planilha.get_worksheet(2)
else:
  if curso == 2:
    # Acesso à página Disciplinas Direito: Pré-Requisitos
    preReq = planilha.get_worksheet(3)

    # Acesso à página Disciplinas Direito
    infoDisc = planilha.get_worksheet(4)

    # Acesso à página Períodos Direito
    infoPeriod = planilha.get_worksheet(5)
  else:
    if curso == 3:
      # Acesso à página Disciplinas SeSo: Pré-Requisitos
      preReq = planilha.get_worksheet(6)

      # Acesso à página Disciplinas SeSo
      infoDisc = planilha.get_worksheet(7)

      # Acesso à página Períodos SeSo
      infoPeriod = planilha.get_worksheet(8)
    else:
      print('Opção Inválida')

In [ ]:
# # Acesso à página Disciplinas EngComp: Pré-Requisitos
# preReq = planilhaEngComp.get_worksheet(0)

In [ ]:
# # Acesso à página Disciplinas EngComp
# infoDisc = planilhaEngComp.get_worksheet(1)

In [ ]:
# # Acesso à página Períodos EngComp
# infoPeriod = planilhaEngComp.get_worksheet(2)

## **Conversão dos Dados da Planilha para DataFrames da Pandas**

In [ ]:
import pandas as pd

In [ ]:
# Conversão da página Disciplinas EngComp: Pré-Requisitos para um DataFrame
df_preReq = pd.DataFrame(preReq.get_all_records())
df_preReq = df_preReq.set_index('Disciplinas')
# df_preReq.index.names = [None]
# df_preReq

In [ ]:
# Conversão da página Disciplinas EngComp para um DataFrame
# df_infoDisc = pd.DataFrame(infoDisc.get_all_records())
df_infoDisc = pd.DataFrame(infoDisc.get_all_records())
df_infoDisc = df_infoDisc.set_index('Dados')
# df_infoDisc.index.names = [None]
# df_infoDisc

In [ ]:
# Conversão da página Períodos EngComp para um DataFrame
# df_infoPeriod = pd.DataFrame(infoPeriod.get_all_records())
df_infoPeriod = pd.DataFrame(infoPeriod.get_all_records())
df_infoPeriod = df_infoPeriod.set_index('Período')
# df_infoPeriod

# **Modelo Matemático**

In [ ]:
%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 29.1 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp
from gurobipy import GRB
from numpy import size

In [ ]:
# Licença Acadêmica
en = gp.Env(empty= True)
en.setParam('WLSACCESSID' , '819a2f40-7f47-432d-bfdc-84a4bc9c39d6' )
en.setParam('WLSSECRET' , '3b0a333f-adcf-4d9a-a8a3-061a18f658ef' )
en.setParam('LICENSEID' , 866515)
en.start()

# Cria o modelo dentro do ambiente Gurobi
m = gp.Model('PO', env=en)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 866515
Academic license - for non-commercial use only - registered to thais.soares@estudante.ufjf.br


## **Variáveis de Decisão**

$x_{i, j}$

onde,

* $i$ é a disciplina
* $j$ é o período

In [ ]:
var = []
varAux = []

disc = []
period = []
period_disc = {}

jInicioEncontrado = False
jInicio = 0

for j in df_infoPeriod.index:
  if df_infoPeriod['Já cursado'][j] == 0:
    period.append(j)
    if jInicioEncontrado == False:
      jInicioEncontrado = True
      jInicio = j

for i in df_infoDisc:
  if i != '' and df_infoDisc[i]['Já cursada'] == 0:
    for j in period:
      adiciona = False
      if df_infoDisc[i]['Período em que é ofertada'] == 0:
        adiciona = True
      else:
        if df_infoDisc[i]['Período em que é ofertada'] == 1 and j % 2 != 0:
          adiciona = True
        else:
          if df_infoDisc[i]['Período em que é ofertada'] == 2 and j % 2 == 0:
            adiciona = True
      varAux.append((i, j))
      if adiciona:
        var.append((i, j))
        if period_disc.get(j, None) == None:
          period_disc[j] = [i]
        else:
          period_disc.get(j, None).append(i)
    disc.append(i)
disc
# var

['MAC005',
 'MAC008',
 'MAC023',
 'DCC001',
 'DCC062',
 'DCC042',
 'DCC063',
 'MAC034',
 'MAC534',
 'MAC035',
 'DCC125',
 'DCC198',
 'MAC021',
 'MAC033',
 'Eletiva 3',
 'Eletiva 4',
 'Eletiva 5',
 'Eletiva 6',
 'Eletiva 7']

In [ ]:
v = m.addVars(var, vtype = GRB.BINARY, name = 'v')

## **Variáveis Auxiliares**

$y_{i, j}$

onde,

* $i$ é uma disciplina que foi alocada antes do período $j$

In [ ]:
y = m.addVars(varAux, vtype = GRB.BINARY, name = 'y')

## **Restrições**

### **1º Restrição**

$∀ i$, $∀ j$  $\quad$  $x_{i, j} = (0, 1)$  

### **2º Restrição**



$∀ i$ $\quad$ $∑\limits_{j} x_{i, j} <= 1$

In [ ]:
r2 = m.addConstrs((v.sum(i, '*') <= 1 for i in disc), name='r2')

### **3º Restrição**

$∀j$ $\quad$ $∑\limits_{i} k_i \ \cdot \ x_{i, j} ≤ M_{j}$

onde,

* $k_i$ é a carga horária da disciplina $i$
* $M_j$ é a carga horária máxima do período $j$

In [ ]:
k = {}
for var in v:
  k[var] = df_infoDisc[var[0]]['Carga Horária']

In [ ]:
r3 = m.addConstrs((gp.quicksum(v[(i, j)] * k[(i, j)] for i in period_disc.get(j, None)) <= df_infoPeriod['Carga Horária Máxima'][j] for j in period), name="r3")

### **4º Restrição**

$∀j$ $\quad$ $∑\limits_{i} t_i \ \cdot \ x_{i, j} < N_{j}$

onde,

* $t_i$ é o grau de dificuldade da disciplina $i$
* $N_j$ é o grau de dificuldade máximo do período $j$

In [ ]:
t = {}
for var in v:
  t[var] = df_infoDisc[var[0]]['Grau de Dificuldade']

In [ ]:
r4 = m.addConstrs((gp.quicksum(v[(i, j)] * t[(i, j)] for i in period_disc.get(j, None)) <= df_infoPeriod['Dificuldade Máxima'][j] for j in period), name="r4")

### **5º Restrição**



Dado Binário $P$:

* $P_{k, i} = 1$, se $k$ é pré-requisito de $i$

* $P_{k, i} = 0$, caso contrário

$∀i, \ ∀j, \ ∀k \ \quad P_{k, i} \cdot x_{i, j} ≤ y_{k, j}$

$\forall k, ∀j \quad \quad \quad  ∑\limits_{l < j} x_{k,l} = y_{k, j}$

onde,

* k é uma disciplina que é pré requisito de i

In [ ]:
P = {}
for d1 in disc:
  for d2 in disc:
    if d1 != d2:
      # d1 é pré-requisito de d2
      if df_preReq[d1][d2] == 1:
          P[(d1, d2)] = 1
      # d1 não é pré-requisito de d2
      else:
          P[(d1, d2)] = 0

for key, value in P.items():
  if value == 1:
    print(key)

('MAC008', 'MAC034')
('MAC008', 'MAC534')
('DCC062', 'DCC125')
('MAC035', 'DCC198')
('MAC021', 'MAC033')


In [ ]:
cont = 0
for varAux in y:
  for var in v:
    if (varAux[0] != var[0] and varAux[1] == var[1] and P.get((varAux[0], var[0])) == 1):
      m.addConstr(v[var] * P.get((varAux[0], var[0])) <= y[varAux], name='r5P1.' + str(cont))
      cont += 1

In [ ]:
cont = 0
for varAux in y:
  for d in disc:
    if (d != varAux[0] and P.get((varAux[0], d)) == 1):
      inicio = jInicio
      passo = 1
      if df_infoDisc[varAux[0]]['Período em que é ofertada'] == 2:
        if jInicio % 2 == 0:
          inicio = jInicio
        else:
          inicio = jInicio + 1
        passo = 2
      else:
        if df_infoDisc[varAux[0]]['Período em que é ofertada'] == 1:
          if jInicio % 2 == 0:
            inicio = jInicio + 1
          else:
            inicio = jInicio
          passo = 2
      # if  varAux[1] > inicio:
      m.addConstr((gp.quicksum(v[(varAux[0], l)] for l in range(inicio, varAux[1], passo)) == y[varAux]), name='r5P2.' + str(cont))
      cont += 1
      break

### **6º Restrição**

$∀i$, $∀j$  $\quad$ $\ x_{i,j} = \ x_{k, j}$

onde,

* $k$ é uma disciplina que deve ser cursada de forma conjunta com $i$, ou seja, no mesmo período $j$

In [ ]:
disc_conj = {}
for d1 in disc:
  for d2 in disc:
    if d1 != d2:
      # d1 e d2 devem ser feitas em um mesmo período
      if df_preReq[d1][d2] != 1 and df_preReq[d1][d2] != 0 and disc_conj.get((d2, d1), None) is None:
          disc_conj[(d1, d2)] = df_preReq[d1][d2]

disc_conj

{('MAC034', 'MAC534'): 4}

In [ ]:
cont = 0
for var1 in v:
  for var2 in v:
    if disc_conj.get((var1[0], var2[0]), None) is not None and var1[1] == var2[1]:
      m.addConstr(v[var1] == v[var2], name='r6.' + str(cont))
      cont += 1

## **Função Objetivo**

$Max \ ∑\limits_{i, \ j}  x_{i, j}$

In [ ]:
m.setObjective(v.sum('*','*'), GRB.MAXIMIZE)

In [ ]:
# Escreve o modelo em um arquivo .lp
m.write('PO - Trabalho 2.lp')

In [ ]:
# Otimiza o modelo
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license - for non-commercial use only - registered to thais.soares@estudante.ufjf.br
Optimize a model with 143 rows, 450 columns and 1022 nonzeros
Model fingerprint: 0xa2be96b9
Variable types: 0 continuous, 450 integer (450 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective 10.0000000
Presolve removed 109 rows and 391 columns
Presolve time: 0.00s
Presolved: 34 rows, 59 columns, 198 nonzeros
Variable types: 0 continuous, 59 integer (59 binary)
Found heuristic solution: objective 17.0000000

Root relaxation: objective 1.900000e+01, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Obj

In [ ]:
qtdd_var_period = {}
var_period = {}

cont = 0

# Imprime em qual período cada disciplina foi alocada
for var in v:
  if v[var].x > 1e-6:
    cont += 1
    if qtdd_var_period.get(var[1], None) is None:
      qtdd_var_period[var[1]] = 1
      var_period[var[1]] = [var[0]]
    else:
      qtdd_var_period[var[1]] += 1
      var_period[var[1]].append(var[0])
    print(v[var].varName, v[var].x)

v[MAC005,8] 1.0
v[MAC008,9] 1.0
v[MAC023,8] 1.0
v[DCC001,9] 1.0
v[DCC062,8] 1.0
v[DCC042,9] 1.0
v[DCC063,11] 1.0
v[MAC034,10] 1.0
v[MAC534,10] 1.0
v[MAC035,8] 1.0
v[DCC125,9] 1.0
v[DCC198,9] 1.0
v[MAC021,8] 1.0
v[MAC033,11] 1.0
v[Eletiva 3,11] 1.0
v[Eletiva 4,10] 1.0
v[Eletiva 5,10] 1.0
v[Eletiva 6,10] 1.0
v[Eletiva 7,10] 1.0


In [ ]:
if cont == size(disc):
  print('Todas as disciplinas foram alocadas em algum período.')
else:
  print('Nem todas as disciplinas foram alocadas em algum período. Reveja os parâmetros de entrada.')

Todas as disciplinas foram alocadas em algum período.


In [ ]:
for var in y:
  if y[var].x > 1e-6:
    print(y[var].varName, y[var].x)

y[MAC008,10] 1.0
y[MAC008,11] 1.0
y[MAC008,12] 1.0
y[MAC008,13] 1.0
y[MAC008,14] 1.0
y[MAC008,15] 1.0
y[MAC008,16] 1.0
y[MAC008,17] 1.0
y[MAC008,18] 1.0
y[MAC008,19] 1.0
y[MAC008,20] 1.0
y[DCC062,9] 1.0
y[DCC062,10] 1.0
y[DCC062,11] 1.0
y[DCC062,12] 1.0
y[DCC062,13] 1.0
y[DCC062,14] 1.0
y[DCC062,15] 1.0
y[DCC062,16] 1.0
y[DCC062,17] 1.0
y[DCC062,18] 1.0
y[DCC062,19] 1.0
y[DCC062,20] 1.0
y[MAC035,9] 1.0
y[MAC035,10] 1.0
y[MAC035,11] 1.0
y[MAC035,12] 1.0
y[MAC035,13] 1.0
y[MAC035,14] 1.0
y[MAC035,15] 1.0
y[MAC035,16] 1.0
y[MAC035,17] 1.0
y[MAC035,18] 1.0
y[MAC035,19] 1.0
y[MAC035,20] 1.0
y[MAC021,9] 1.0
y[MAC021,10] 1.0
y[MAC021,11] 1.0
y[MAC021,12] 1.0
y[MAC021,13] 1.0
y[MAC021,14] 1.0
y[MAC021,15] 1.0
y[MAC021,16] 1.0
y[MAC021,17] 1.0
y[MAC021,18] 1.0
y[MAC021,19] 1.0
y[MAC021,20] 1.0


# **Visualização dos Resultados**

In [ ]:
%pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 9.8 MB/s eta 0:00:00


In [ ]:
import xlsxwriter
import string

In [ ]:
def linha_period (j):
  if j == jInicio:
    return [1, 2]
  if j == jInicio + 1:
    return [3, 4]
  if j == jInicio + 2:
    return [5, 6]
  if j == jInicio + 3:
    return [7, 8]
  if j == jInicio + 4:
    return [9, 10]
  if j == jInicio + 5:
    return [11, 12]
  if j == jInicio + 6:
    return [13, 14]
  if j == jInicio + 7:
    return [15, 16]
  if j == jInicio + 8:
    return [17, 18]
  if j == jInicio + 9:
    return [19, 20]
  if j == jInicio + 10:
    return [21, 22]
  if j == jInicio + 11:
    return [23, 24]
  if j == jInicio + 12:
    return [25, 26]
  if j == jInicio + 13:
    return [27, 28]
  if j == jInicio + 14:
    return [29, 30]
  if j == jInicio + 15:
    return [31, 32]
  if j == jInicio + 16:
    return [33, 34]
  if j == jInicio + 17:
    return [35, 36]
  if j == jInicio + 18:
    return [37, 38]
  if j == jInicio + 19:
    return [39, 40]

In [ ]:
workbook = xlsxwriter.Workbook('PO - Trabalho 2.xlsx')
worksheet = workbook.add_worksheet('Visualização')
format = workbook.add_format({'align':'center','valign':'vcenter', 'border':1, 'bg_color': '#B0C4DE'})
format1 = workbook.add_format({'align':'center','valign':'vcenter', 'border':1, 'bg_color': '#f2f2f2'})

worksheet.hide_gridlines(2)

for j, qtdd_disc in qtdd_var_period.items():
  linhas = linha_period(j)
  worksheet.merge_range('A' + str(linhas[0]) + ':B' + str(linhas[1]), str(j) + 'º' + ' Período', format)
  colunas = list(string.ascii_uppercase[2: (2 + qtdd_disc * 2)])
  coluna = 0
  for i in var_period.get(j):
    worksheet.merge_range(colunas[coluna] + str(linhas[0]) + ':' + colunas[coluna + 1] + str(linhas[1]), i, format1)
    coluna += 2

workbook.close()

In [ ]:
m.dispose()
en.dispose()

# **Referências**

https://colab.research.google.com/notebooks/io.ipynb#scrollTo=ASdgLIt2s-ux

https://support.gurobi.com/hc/en-us/articles/4409582394769-Google-Colab-Installation-and-Licensing

https://acervolima.com/como-converter-a-coluna-dataframe-em-um-indice-no-python-pandas/

https://www.gurobi.com/